In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
tqdm.pandas()

## Общая предобработка

In [2]:
data1 = pd.read_csv('internship_clickstream_data_1.gzip', compression='gzip')
data2 = pd.read_csv('internship_clickstream_data_2.gzip', compression='gzip')
data3 = pd.read_csv('internship_clickstream_data_3.gzip', compression='gzip')
print(data1.shape, data2.shape, data3.shape)

(30000000, 8) (30000000, 8) (22463346, 8)


In [3]:
# drop duplicates
data1.drop_duplicates(subset=['uid', 'offer_id'], inplace=True)
data2.drop_duplicates(subset=['uid', 'offer_id'], inplace=True)
data3.drop_duplicates(subset=['uid', 'offer_id'], inplace=True)
print(data1.shape, data2.shape, data3.shape)

# work with date
data1['timestamp'] = pd.to_datetime(data1['timestamp'])
data2['timestamp'] = pd.to_datetime(data2['timestamp'])
data3['timestamp'] = pd.to_datetime(data3['timestamp'])

# save users from 1 week
data1_s = data1[data1['timestamp'] < pd.to_datetime('2022-07-10 00:00:00.020000')]
data2_s = data2[data2['timestamp'] < pd.to_datetime('2022-07-10 00:00:00.020000')]
data3_s = data3[data3['timestamp'] < pd.to_datetime('2022-07-10 00:00:00.020000')]
users_1_week = pd.concat([data1_s, data2_s, data3_s])['uid'].unique()

# filter by date, third week
data1 = data1[(data1['timestamp'] >= pd.to_datetime('2022-07-17 00:00:00.020000')) & \
              (data1['timestamp'] < pd.to_datetime('2022-07-24 00:00:00.020000'))]
data2 = data2[(data2['timestamp'] >= pd.to_datetime('2022-07-17 00:00:00.020000')) & \
              (data2['timestamp'] < pd.to_datetime('2022-07-24 00:00:00.020000'))]
data3 = data3[(data3['timestamp'] >= pd.to_datetime('2022-07-17 00:00:00.020000')) & \
              (data3['timestamp'] < pd.to_datetime('2022-07-24 00:00:00.020000'))]
print(data1.shape, data2.shape, data3.shape)

# take users from 3 week who is not in 1 week
data1 = data1[~data1['uid'].isin(users_1_week)]
data2 = data2[~data2['uid'].isin(users_1_week)]
data3 = data3[~data3['uid'].isin(users_1_week)]

data = pd.concat([data1, data2, data3])
print('after concating:', data.shape)
data.drop_duplicates(subset=['uid', 'offer_id'], inplace=True)
print('after drop duplicates:', data.shape)

(22585345, 8) (22559623, 8) (17427910, 8)
(5105336, 8) (5365398, 8) (4175515, 8)
after concating: (6278207, 8)
after drop duplicates: (5452078, 8)


In [4]:
# take users that have > 5 clicks
while not (data['offer_id'].value_counts()[data['offer_id'].value_counts() <= 5].empty) or \
      not (data['uid'].value_counts()[data['uid'].value_counts() <= 5].empty):
    offer_ids = data['offer_id'].value_counts()[data['offer_id'].value_counts() > 5].index
    data = data[data['offer_id'].isin(offer_ids)]
    uids = data['uid'].value_counts()[data['uid'].value_counts() > 5].index
    data = data[data['uid'].isin(uids)]
print(data.shape)

(2707705, 8)


## Добавляем lst-features
Чтобы выделить объявления для аренды и продажи

In [6]:
data11 = pd.read_csv('lst_announcement_data_1.gzip', compression='gzip', usecols=['id', 'category'])
data22 = pd.read_csv('lst_announcement_data_2.gzip', compression='gzip', usecols=['id', 'category'])
data33 = pd.read_csv('lst_announcement_data_3.gzip', compression='gzip', usecols=['id', 'category'])

In [7]:
print(data11.shape, data22.shape, data33.shape)
data11.drop_duplicates(subset=['id'], inplace=True)
data22.drop_duplicates(subset=['id'], inplace=True)
data33.drop_duplicates(subset=['id'], inplace=True)
print(data11.shape, data22.shape, data33.shape)

lst_features = pd.concat([data11, data22, data33])
print('after concating:', lst_features.shape)
lst_features.drop_duplicates(subset=['id'], inplace=True)
print('after drop duplicates:', lst_features.shape)

(1000000, 2) (1000000, 2) (778005, 2)
(1000000, 2) (1000000, 2) (778005, 2)
after concating: (2778005, 2)
after drop duplicates: (2778005, 2)


In [8]:
print(lst_features['category'].unique())

['townhouseSale' 'commercialLandRent' 'landSale' 'flatRent' 'flatSale'
 'houseSale' 'dailyHouseRent' 'newBuildingFlatSale' 'dailyFlatRent'
 'shoppingAreaSale' 'cottageRent' 'roomRent' 'freeAppointmentObjectRent'
 'businessSale' 'cottageSale' 'officeRent' 'freeAppointmentObjectSale'
 'townhouseRent' 'garageSale' 'houseShareSale' 'industryRent' 'officeSale'
 'roomSale' 'industrySale' 'warehouseRent' 'commercialLandSale'
 'houseRent' 'houseShareRent' 'garageRent' 'shoppingAreaRent'
 'buildingSale' 'dailyRoomRent' 'buildingRent' 'warehouseSale' 'bedRent'
 'flatShareSale' 'dailyBedRent' 'businessRent']


In [9]:
def sale_rent(arg):
    """
        кодируем объявления продажи 1
        аренды - 0
    """
    if 'Rent' in arg:
        return 0
    if 'Sale' in arg:
        return 1
    else:
        return np.nan
lst_features['category'] = lst_features['category'].progress_apply(sale_rent)
lst_features['category'].unique()

100%|█████████████████████████████| 2778005/2778005 [00:03<00:00, 826217.96it/s]


array([1, 0])

In [10]:
merged = data.merge(lst_features, left_on='offer_id', right_on='id', how='left')
print(merged.shape)
merged.head()

(2707705, 10)


,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,category
0,2022-07-21 14:59:12.005,035ac9e59f264efc,93558783,ios,OpenOfferScreen,OfferScreen,200902594,2022-07-21,200902594.0,0.0
1,2022-07-21 14:59:32.553,fd4ecea17e5348ab,275A277B-6BB1-48EE-9CF4-38530F69EBB0,ios,OpenOfferScreen,SearchResultsList,267412812,2022-07-21,267412812.0,1.0
2,2022-07-21 00:00:13.118,c201728987f846ff,84210581,ios,OpenOfferScreen,MapScreen,244887246,2022-07-21,244887246.0,1.0
3,2022-07-22 23:59:27.313,3ea3b2c9449f4e7c,93644412,ios,OpenOfferScreen,FavoritesScreen,276242581,2022-07-22,276242581.0,0.0
4,2022-07-22 23:59:44.353,95b8ca8b8b124f30,6fd68d4e-b82e-4015-9b0a-d9fc2d1106a5,android,OpenOfferScreen,SearchResultsList,233845560,2022-07-22,233845560.0,0.0


In [11]:
grouped = merged.groupby(['uid'])['category'].agg(['count', 'sum']).reset_index()
grouped

,uid,count,sum
0,0000AEA9-B496-418E-8635-B8326AB61211,11,0.0
1,00014a6e-0ea8-48e6-a761-cecba18cdd5c,17,15.0
2,0001ACBB-6FBE-4E41-B824-0771C7E7276D,35,0.0
3,00028c8b-7f19-42a5-84f1-167d4b632b04,21,0.0
4,0002a451-ec78-40ea-8c7b-4a55bb6ef1b7,9,0.0
...,...,...,...
150938,fff7d627-688a-40b3-9ba2-e5c565c10550,16,16.0
150939,fff8109f-1016-426f-b942-a6182868b5df,27,0.0
150940,fffa5f0b-d9ea-40f5-8d4e-a3a29c290dc9,8,8.0
150941,fffca2a3-8ad7-4f4e-a2ab-28dd7744f899,6,0.0


In [12]:
users = data['uid'].unique()
grouped['per'] = grouped['sum'] / grouped['count']

def enc_pref(arg):
    """
        кто больше интересуется продажей - 1
        кто больше интересуется арендой - 0
    """
    if arg > 0.5:
        return 1
    else:
        return 0
grouped['pref'] = grouped['per'].progress_apply(enc_pref)
grouped['pref']

100%|███████████████████████████████| 150943/150943 [00:00<00:00, 662230.36it/s]


0         0
1         1
2         0
3         0
4         0
         ..
150938    1
150939    0
150940    1
150941    0
150942    0
Name: pref, Length: 150943, dtype: int64

In [13]:
grouped['pref'].to_csv('uid_3_week_preferences.csv')